# Local Postgresql Database Lab

In this lab we will practice adding and removing data from a local Postgresql database. We will:

- Walk through instructions on how to set up and configure a local postgresql server
- Create a database and a table using SQL and CRUD data in/from that table.
- Learn how to copy data from a CSV file into this local database
- Practice our SQL with a practice assignment.

## 1. Install postgresql locally

You should have already taken care of this in the Installfest, but just in case, [here](http://www.postgresql.org/download/) are the instructions for the most common platforms.

## 2. Launch and connect to a local PostgreSQL server

Now that you've installed PostgreSQL locally, practice connecting with the following methods:

- [Postico](https://eggerapps.at/postico/docs/v1.0.6/)
- Command-line (psql)
- Sqlalchemy + Pandas
- Ipython-notebook using the ipython-sql extension

**Check:** List the existing databases, there should be none, right?
> not true, there are some default databases from postgres

**Check:** If there are databases, check the tables' contents.

In [1]:
from sqlalchemy import create_engine
import pandas as pd
engine = create_engine('postgresql://localhost:5432')
pd.read_sql("SELECT * FROM information_schema.tables LIMIT 3;", engine)

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
0,prerana,pg_catalog,pg_statistic,BASE TABLE,None,None,None,None,None,YES,NO,None
1,prerana,pg_catalog,pg_type,BASE TABLE,None,None,None,None,None,YES,NO,None
2,prerana,pg_catalog,pg_authid,BASE TABLE,None,None,None,None,None,YES,NO,None


In [2]:
%load_ext sql

In [3]:
%%sql postgresql://localhost:5432
SELECT * FROM information_schema.tables LIMIT 3;        

3 rows affected.


table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
prerana,pg_catalog,pg_statistic,BASE TABLE,None,None,None,None,None,YES,NO,None
prerana,pg_catalog,pg_type,BASE TABLE,None,None,None,None,None,YES,NO,None
prerana,pg_catalog,pg_authid,BASE TABLE,None,None,None,None,None,YES,NO,None


## 3. Create DB
Once you are connected to your local Postgresql server, create a new database. Call it `lab33`.

Practice creating and deleting the database with the following methods:

- Postico
- Command-line:
>     create database lab33
>     drop database lab33

- Sqlalchemy
- Ipython-notebook using the ipython-sql extension

**Check:** If you create a DB in Postico, do you see it from the command-line?
> Yes

**Check:** What does this tell us about multiple connections to the same db?
> Multiple users/connections are possible and the data is consistent. Postgres is relational -> ACID.

In [6]:
%%sql
create database lab33



InternalError: (psycopg2.InternalError) CREATE DATABASE cannot run inside a transaction block
 [SQL: 'create database lab33']

## 4. CRUD (Create, Read, Update, and Delete)

In lesson 1.1 you learned how to add and remove data from sqlite. Let's review that on our local PostgreSQL installation.

### 4.a: Creating Tables and Adding Columns

1. Create an table called `table1` with a single column `field1` containing an INTEGER PRIMARY KEY. Practice doing this with any of the methods above.
- Add a few more columns to `table1`:
    - field2 VARCHAR(16)
    - field3 REAL
    - field4 TEXT
check [the doc](http://www.postgresql.org/docs/9.3/static/datatype.html) for more info on data types supported by postgresql.
- Check tables and schemas using command line or postico

In [7]:
%%sql postgresql://localhost:5432/lab33
        
CREATE TABLE table1(field1 INTEGER PRIMARY KEY) ;


(psycopg2.ProgrammingError) relation "table1" already exists
 [SQL: 'CREATE TABLE table1(field1 INTEGER PRIMARY KEY) ;']


In [8]:
%%sql
ALTER TABLE table1 ADD column field2 VARCHAR(16); 
ALTER TABLE table1 ADD column field3 REAL;
ALTER TABLE table1 ADD column field4 TEXT;

(psycopg2.ProgrammingError) column "field2" of relation "table1" already exists
 [SQL: 'ALTER TABLE table1 ADD column field2 VARCHAR(16);']


In [9]:
%%sql
select * from table1;

2 rows affected.


field1,field2,field3,field4
1,Henry James,42.0,"75 Mission Street, San Francisco, CA"
2,Carol James,40.0,"75 Mission Street, San Francisco, CA"


In [10]:
%%sql

select * from table1;

2 rows affected.


field1,field2,field3,field4
1,Henry James,42.0,"75 Mission Street, San Francisco, CA"
2,Carol James,40.0,"75 Mission Street, San Francisco, CA"


### 4.b: Add Data

Add some data to `table1`:

|field1|field2|field3|field4|
|----|
|1|'Henry James'|42|'75 Mission Street, San Francisco, CA'|
|2|'Carol James'|40|'75 Mission Street, San Francisco, CA'|
|3|'Jesse James'|12|'75 Mission Street, San Francisco, CA'|

### 4.c: Read Data
Read the content of the table,

In [11]:
%%sql
select * from table1;


2 rows affected.


field1,field2,field3,field4
1,Henry James,42.0,"75 Mission Street, San Francisco, CA"
2,Carol James,40.0,"75 Mission Street, San Francisco, CA"


### 4.d: Update Records
Suppose we need to update an existing record with new data - e.g. maybe `Jesse James` is only 9. Use the update command to do this.

In [12]:
%%sql
update table1 set field3 = 3 where field1 = 4;
select * from table1;



0 rows affected.
2 rows affected.


field1,field2,field3,field4
1,Henry James,42.0,"75 Mission Street, San Francisco, CA"
2,Carol James,40.0,"75 Mission Street, San Francisco, CA"


### 4.e: Remove Records
To remove records use the DELETE command. Delete the entry for anyone matching `Jesse`.

In [13]:
%%sql
delete from table1 where field1 = 4;


0 rows affected.


[]

## 5. Data from CSV

Copy CSV data from a local file into a local PostgreSQL database. We will use a [dataset](../../assets/datasets/Eviction_Notices.csv) pulled from the SF open data website. It contains a set of eviction notices issued in San Francisco.

Open the data in a text editor and have a look at it.

### 5.a: Simple Import

Read [this post](http://stackoverflow.com/questions/2987433/how-to-import-csv-file-data-into-a-postgresql-table) to learn how you can import data from CSV to PostgreSQL.

Notice that you have to decide the type for each column. Let's keep it simple and import everything as a string of text for now.

- Create a table called `evictions_simple` and import the data as varchar.
- Check that everything worked fine by loading a few lines from the table.
- Try some simple queries:
    - count how many evictions are due to non_payments
    - list the 3 most recent evictions


**Hint:** A good practice when you create a new table is to always drop it first, in case it already exists.

**Check:** Did you avoid importing the header as a record?

In [14]:
import psycopg2

In [27]:
%%sql
DROP TABLE IF EXISTS evictions_simple;
CREATE TABLE evictions_simple
(eviction_id varchar,
 address varchar,
 city varchar,
 state varchar,
 zip varchar,
 file_date varchar,
 non_payment varchar,
 breach varchar,
 nuisance varchar,
 illegal_use varchar,
 failure_to_sign_renewal varchar,
 access_denial varchar,
 unapproved_subtenant varchar,
 owner_move_in varchar,
 demolition varchar,
 capital_improvement varchar,
 substantial_rehab varchar,
 ellis_act_withdrawal varchar,
 condo_conversion varchar,
 roommate_same_unit varchar,
 other_cause varchar,
 late_payments varchar,
 lead_remediation varchar,
 development varchar,
 good_samaritan_ends varchar,
 constraints varchar,
 constraints_date varchar,
 supervisor_district varchar,
 neighborhood varchar,
 client_location varchar);



Done.
Done.


[]

In [28]:
pwd

u'/Users/prerana/Desktop/dsi-prerana/week-05/3.3-lab/code/starter-code'

In [29]:
ls -lrt

total 16536
-rw-r--r--  1 prerana  staff  8416325 Nov 27 10:08 Eviction_Notices.csv
-rw-r--r--  1 prerana  staff    47690 Nov 27 10:48 starter-code-3_3.ipynb


In [32]:
import psycopg2
conn = psycopg2.connect( dbname='postgres', user='postgres')
cur = conn.cursor()
cur.execute("""Copy postgres.evictions_simple from '/Users/prerana/Desktop/dsi-prerana/week-05/3.3-lab/code/starter-code/Eviction_Notices.csv';""")
conn.commit()
conn.close()


ProgrammingError: schema "postgres" does not exist


In [35]:
ls -ltr

total 16544
-rw-r--r--  1 prerana  staff  8416325 Nov 27 10:08 Eviction_Notices.csv
-rw-r--r--  1 prerana  staff    49496 Nov 27 10:54 starter-code-3_3.ipynb


In [37]:
%%sql
COPY evictions_simple FROM 'Eviction_Notices.csv' DELIMITER ',' CSV HEADER;

(psycopg2.OperationalError) could not open file "Eviction_Notices.csv" for reading: No such file or directory
 [SQL: "COPY evictions_simple FROM 'Eviction_Notices.csv' DELIMITER ',' CSV HEADER;"]


In [20]:
%%sql
COPY evictions_simple FROM 'Evictions_Notices';


(psycopg2.OperationalError) could not open file "Evictions_Notices" for reading: No such file or directory
 [SQL: "COPY evictions_simple FROM 'Evictions_Notices' ;"]


In [21]:
%%sql
select * from evictions_simple limit 3;

0 rows affected.


eviction_id,address,city,state,zip,file_date,non_payment,breach,nuisance,illegal_use,failure_to_sign_renewal,access_denial,unapproved_subtenant,owner_move_in,demolition,capital_improvement,substantial_rehab,ellis_act_withdrawal,condo_conversion,roommate_same_unit,other_cause,late_payments,lead_remediation,development,good_samaritan_ends,constraints,constraints_date,supervisor_district,neighborhood,client_location


In [22]:
%%sql
select count(*) from evictions_simple;
where non_payment = 'true';

1 rows affected.
(psycopg2.ProgrammingError) syntax error at or near "where"
LINE 1: where non_payment = 'true';
        ^
 [SQL: "where non_payment = 'true';"]


### 5.b: Data Cleaning and Import

If you've executed the last query correctly (most recent evictions), you'll have noticed that the dates are not correctly understood. This is because we were sloppy and imported the data as string for all fields.

Let's see what data types we would ideally like to have for each column.

[Here are data types](http://www.tutorialspoint.com/postgresql/postgresql_data_types.htm)

- Discuss in pairs each column and then let's summarize together. Which data type would you choose for each field?
- Repeat the import to a new table called `evictions`. Notice that line 31494 may throw an error. Why is that?
> There are two ` characters that do not belong.
- Repeat the query for the 3 most recent evictions. Does it work now?

## 6. Queries

Now that we have imported the data with correct data types, let's query the `evictions` table and find out a few things about SF and evictions.

Questions:
- How many neighborhoods are there in SF? List them alphabetically
- How many supervisor districts? 
- How many unique zip codes?
    - Are there any bad data in these? how many?
- What are the top 5 causes of eviction?

### 6.a: How many neighborhoods are there in SF? List them alphabetically.

### 6.b: How many supervisor districts?

### 6.c how many unique zip codes?

Are there any bad data in these?

### 6.d: What are the top 5 causes of eviction?

You may find it easier to answer this question using pandas.

## Bonus
- Let's count the number of evictions for each year
    - How has the number varied?
    - Can you compare this with the nasdaq index? (use the DataReader module in Pandas to get the data)
